In [6]:
import os, sys, time
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import cv2
from segment_anything import (SamAutomaticMaskGenerator, SamPredictor, sam_model_registry)
import imageio
from utils.segment_utils import *
from scene.neural_3D_dataset_NDC import Neural3D_NDC_Dataset
from IPython.display import clear_output as clear

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model_type = 'vit_h'
sam = sam_model_registry[model_type](checkpoint='/data/sxj/dependencies/sam_ckpt/sam_vit_h_4b8939.pth').to('cuda')
predictor = SamPredictor(sam)

In [3]:
datadir = "data/dynerf/cut_roasted_beef"
camdir = "cam00"
imgdir = os.path.join(datadir, camdir, "images")
images_path = os.listdir(imgdir)
images_path.sort()

In [12]:
with torch.no_grad():
    for i in range(263, len(images_path)):
        # sys.stdout.write('\r')
        # sys.stdout.write("Reading image {}/{}".format(i+1, len(images_path)))
        # sys.stdout.flush()
        print("Reading image {}/{}".format(i+1, len(images_path)))
        
        img = Image.open(os.path.join(imgdir, images_path[i]))
        img = np.array(img)
        
        input_point = np.array([[750, 700], [800, 500]])
        # input_point = np.array([[700, 600]])
        input_label = np.ones(len(input_point))

        predictor.set_image(img)
        vanilla_masks, scores, logits = predictor.predict(
            point_coords=input_point,
            point_labels=input_label,
            multimask_output=True,
        )

        mask_id = np.argmax(scores)
        plt.imshow(img)
        show_points(input_point, input_label, plt.gca())
        show_mask(vanilla_masks[mask_id], plt.gca())
        plt.title(f"Mask {mask_id}, Score: {scores[mask_id]:.3f}", fontsize=18)
        plt.show()
        time.sleep(2)

        save_dir = os.path.join(datadir, camdir, "gt_mask", "man")
        os.makedirs(save_dir, exist_ok=True)
        cv2.imwrite(os.path.join(save_dir, images_path[i]), vanilla_masks[mask_id].astype("int"))
        
        clear()